In [1]:
!pip install flask-ngrok
!pip install pyngrok

In [2]:
from flask import Flask, request, render_template, url_for
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import os
from werkzeug.utils import secure_filename



In [3]:
!mkdir -p /content/static/uploads

In [4]:
html_code = '''
<!DOCTYPE html>
<html>
<head>
    <title>Snake Classifier</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin-top: 50px;
        }
        .upload-box {
            border: 2px dashed #aaa;
            padding: 40px;
            width: 400px;
            margin: auto;
            background-color: #f9f9f9;
        }
        .upload-box input {
            margin-top: 20px;
        }
        .search-button {
            margin-top: 20px;
            padding: 10px 20px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
        }
    </style>
</head>
<body>
    <h1>Snake Classifier</h1>
    <form action="/predict" method="POST" enctype="multipart/form-data">
        <div class="upload-box">
            <p>Drag image here or upload file</p>
            <input type="file" name="image">
            <br>
            <button class="search-button" type="submit">Search</button>
        </div>
    </form>
</body>
</html>
'''

os.makedirs("templates", exist_ok=True)
with open("templates/index.html", "w") as f:
    f.write(html_code)

In [5]:
html_code = '''
<!DOCTYPE html>
<html>
<head>
    <title>Snake Classifier - Result</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            padding: 30px;
        }
        .image-row {
            display: flex;
            justify-content: space-between;
        }
        .image-box {
            width: 45%;
            border: 1px solid #ccc;
            padding: 10px;
            text-align: center;
        }
        .info-grid {
            display: grid;
            grid-template-columns: 1fr 1fr;
            margin-top: 30px;
            gap: 20px;
        }
        .info-grid .left, .info-grid .right {
            padding: 10px;
            border-top: 1px solid #ccc;
        }
        h3 {
            margin-bottom: 5px;
        }
        .field-label {
            font-weight: bold;
        }
    </style>
</head>
<body>

<h1>Snake Classifier</h1>

<div class="image-row">
    <div class="image-box">
        <h3>Wiki Image</h3>
        <img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Aspidites_melanocephalus.jpg"
             alt="Wiki Image" width="100%">
    </div>
    <div class="image-box">
        <h3>Uploaded Image</h3>
        <img src="{{ uploaded_image_url }}" alt="Uploaded" width="100%">
    </div>
</div>

<div class="info-grid">
    <div class="left">
        <p><span class="field-label">NAME:</span> Black-headed Python</p>
        <p><span class="field-label">LATIN NAME:</span> Aspidites melanocephalus</p>
        <p><span class="field-label">DESCRIPTION:</span> Non-venomous python species found in northern Australia.</p>
        <p><span class="field-label">HABITAT:</span> Savannahs, dry forests, and scrublands</p>
        <p><span class="field-label">BEHAVIOR:</span> Mostly terrestrial, non-aggressive</p>
        <p><span class="field-label">POISONOUS:</span> NO</p>
        <p><span class="field-label">DANGER LEVEL:</span> Low</p>
    </div>
    <div class="right">
        <h3>Emergency Treatment:</h3>
        <p>This species is non-venomous and typically does not require medical attention unless infection occurs from bite wounds.</p>
    </div>
</div>

</body>
</html>
'''

os.makedirs("templates", exist_ok=True)
with open("templates/result.html", "w") as f:
    f.write(html_code)

In [11]:
# 这里你们要注册一个ngrok的号，然后在终端里输入下面的命令
# ngrok config add-authtoken 你们自己的验证码

app = Flask(__name__)
UPLOAD_FOLDER = '/content/static/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return 'No file uploaded', 400
    file = request.files['image']
    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(filepath)

    uploaded_url = url_for('static', filename=f'uploads/{filename}')

    return render_template('result.html', uploaded_image_url=uploaded_url)

public_url = ngrok.connect(5000)
print("Public Network Address:", public_url)

# initial flask
app.run()

Public Network Address: NgrokTunnel: "https://6fdd-34-106-154-190.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [12]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return 'No file uploaded', 400
    file = request.files['image']
    filename = secure_filename(file.filename)
    file.save(os.path.join('/content', filename))
    return f'Image received: {filename}（Model not conntected）', 200

public_url = ngrok.connect(5000)
print("Public Network Address: ", public_url)

app.run()

Public Network Address:  NgrokTunnel: "https://de7b-34-106-154-190.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
